In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

pmapp = mapp[mapp.biotype == 'protein_coding'].copy()

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

In [4]:
import os
network_files = os.listdir('output')

def network_genes(genes):
    all_connectors = []
    i = 0
    for gene1 in genes:
#         if i % 100 == 0:
#             print('{}/{}'.format(i, len(genes)))

        i = i + 1
        
        if '{}_nearest_neighbor_weighted.txt'.format(gene1) not in network_files:
            continue
        net = pd.read_csv('output/{}_nearest_neighbor_weighted.txt'.format(gene1), sep='\t')
        net = net.set_index('End gene', drop=False)
        end_genes = net['End gene'].to_list()

        for gene2 in genes:
            if gene2 not in end_genes:
                continue
            connectors = net.loc[gene2, 'Connector genes']

            if (type(connectors) == float) or (type(connectors) == np.float64):
                continue

            all_connectors = all_connectors + connectors.split(',')

        all_connectors = list(set(all_connectors))
        
    return all_connectors

In [5]:
outs = pd.read_csv('Supp_table_all_variants.SVs_included.csv')

In [6]:
snv = pd.read_csv('16p_all_snvs.csv', sep=',')

def str2float(s, default = 0):
    if s == '.':
        return default
    else:
        return float(s)

    
snv.CADD = snv.CADD.apply(lambda s: str2float(s))

lof = snv.Mut_type == 'lof'
cadd_10 = snv.CADD > 10

snv = snv[lof | cadd_10].copy()

In [7]:
cnv = pd.read_csv('../../cnv_parse/output/merged/16p_all_rare_cnvs.tsv', sep='\t')
cnv = cnv[cnv.gene_covered == 'encapsulated']

In [8]:
net = pd.read_csv('output/{}_nearest_neighbor_weighted.txt'.format('UQCRC2'), sep='\t')
net = net.set_index('End gene', drop=False)
end_genes = net['End gene'].to_list()
ref = end_genes

In [9]:
with open('genes_in_netowrk.txt', 'w') as f:
    for s in end_genes:
        f.write(s)
        f.write('\n')

In [ ]:
for sub in outs['sample'].unique():

    variant_genes = snv[snv['Sample'] == sub]
    variant_genes = list(variant_genes['Gene.refGene'].unique())
    cnv_genes = cnv[cnv.subject == sub].gene.to_list()
    variant_genes = list(set(variant_genes + cnv_genes))
    
    outliers = outs[outs['sample'] == sub].copy()
    outliers = outliers[outliers.outlier == 'X'].gene.to_list()
    
    genes = variant_genes + outliers
    
    connectors = network_genes(outliers)
    print(sub, len(connectors))
    with open('cohort_connectors/{}.outlier_and_rare_variant_connectors.txt'.format(sub), 'w') as f:
        for gene in connectors:
            f.write(gene)
            f.write('\n')

SG001 1027
SG003 244
SG006 1522
SG007 3056
